## Setup

In [30]:
EXPERIMENT_NAME = 'reusable-component-training'

## Define the pipeline

In [31]:
import kfp
from kfp import compiler
import kfp.compiler as compiler
import kfp.components as comp
import kfp.dsl as dsl
from kfp import gcp

In [40]:
concat_operation = kfp.components.load_component_from_url(
    'https://raw.githubusercontent.com/SaschaHeyer/Machine-Learning-Training/master/kubeflow/reusable-component-training/concat/component.yaml')
help(concat_operation)

reverse_operation = kfp.components.load_component_from_url(
    'https://raw.githubusercontent.com/SaschaHeyer/Machine-Learning-Training/master/kubeflow/reusable-component-training/reverse/component.yaml')
help(reverse_operation)

Help on function Training - Concat Text:

Training - Concat Text(first:'String', second:'String')
    Takes two strings and concats them

Help on function Training - Reverse Text:

Training - Reverse Text(text:'String')
    Takes text and reverse its content



## Build the Pipeline Function

In [33]:

@dsl.pipeline( # dsl decoreator
  name='Text transformation pipeline',
  description='A pipeline which concats and reverse strings'
)
def text_transformation_pipeline(first, second):
    
    ## concat the strings
    concat = concat_operation(
        first=first,
        second=second
    ).apply(kfp.gcp.use_gcp_secret('user-gcp-sa')) 
    
    ## reverse the outout of the previous component
    reverse = reverse_operation(
        text=concat.output
    ).apply(kfp.gcp.use_gcp_secret('user-gcp-sa')) 
    
    

# Compile the Pipeline

In [34]:
pipeline_func = text_transformation_pipeline
pipeline_filename = pipeline_func.__name__ + '.pipeline.zip'

compiler.Compiler().compile(text_transformation_pipeline, 
                            pipeline_filename)

# Create a Kubeflow Experiment
Get or create an experiment and submit a pipeline run

In [39]:
client = kfp.Client()

try:
    experiment = client.get_experiment(experiment_name=EXPERIMENT_NAME)
except:
    experiment = client.create_experiment(EXPERIMENT_NAME)
    
print(experiment)

{'created_at': datetime.datetime(2019, 6, 26, 14, 24, 19, tzinfo=tzlocal()),
 'description': None,
 'id': '270e9ad8-5118-41a2-91ea-3cbc05d8b054',
 'name': 'reusable-component-training'}


# Run the Pipeline

In [37]:
#Specify pipeline argument values
arguments = {'first': 'dess', 'second': 'erts'}

run_name = pipeline_func.__name__ + ' run'
run_result = client.run_pipeline(experiment.id, 
                                 run_name, 
                                 pipeline_filename, 
                                 arguments)

print(experiment.id)
print(run_name)
print(pipeline_filename)
print(arguments)

270e9ad8-5118-41a2-91ea-3cbc05d8b054
text_transformation_pipeline run
text_transformation_pipeline.pipeline.zip
{'first': 'dess', 'second': 'erts'}
